In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import yaml
from sklearn.metrics import auc

import sys
main_path = "../.."
sys.path.append(main_path)

In [2]:
model_scores_path = f"{main_path}/data/processed/model_scores"
os.makedirs(model_scores_path, exist_ok=True)

with open(f"{model_scores_path}/best_scores.pkl", 'rb') as f:
    metric_results = pickle.load(f)
    
with open(f"{model_scores_path}/colonflag_scores.pkl", 'rb') as f:
    cf_metric_results = pickle.load(f)

with open(f"{main_path}/config.yaml", 'r') as f:
    config = yaml.safe_load(f)
    
models = ['LR', 'RF', 'SVM']

In [3]:
mean_fpr = np.linspace(0, 1, 100)
def find_mean_auc(result):
    tprs = [i['tpr'] for i in result]
    mean_tpr = np.array(tprs).mean(axis=0)
    mean_tpr[-1] = 1.0
    auc_mean = round(auc(mean_fpr, mean_tpr), 2)
    
    return auc_mean, mean_tpr

In [4]:
# approach, model, f1, auc
best_metric_results = []
mean_tprs = {}

for appr in config['approach_names']:
    best_model = [0, None]
    for m in models:
        score = np.mean(metric_results[appr.lower()][m.lower()]['f1_scores'])
        if score > best_model[0]:
            best_model[0] = score
            best_model[1] = m
    
    result = metric_results[appr.lower()][best_model[1].lower()]['rocauc']
    auc_mean, mean_tpr = find_mean_auc(result)
    
    best_metric_results.append([appr, best_model[1], best_model[0], auc_mean])
    mean_tprs[appr] = {}
    mean_tprs[appr][best_model[1]] = mean_tpr
    
    # colonflag
    f1_mean = np.mean(cf_metric_results[appr.lower()]['f1_scores'])
    
    result = cf_metric_results[appr.lower()]['rocauc']
    auc_mean, mean_tpr = find_mean_auc(result)
    
    best_metric_results.append([appr, 'CF', f1_mean, auc_mean])
    mean_tprs[appr]['CF'] = mean_tpr

In [5]:
results_df = pd.DataFrame(best_metric_results, columns=['approach', 'model', 'f1_score', 'auc_score'])
results_df.to_csv(f'{model_scores_path}/approach_model_f1_auc.csv', index=False)

with open(f'{model_scores_path}/mean_tprs_comparison.pkl', 'wb') as f:
    pickle.dump(mean_tprs, f)